Remove top level module variables from the stdlib sys module.
This is based of the types detected on the sys module in a fimrware stub for esp32 micropython

the approach is very simple, we just comment the top level variables from the sys module
this assumes that all defenitions are on a single line, which may or may not be true

A more mature approach would use cstlib to parse the module and remove the variables


In [5]:
from typing import Set
import re
from pathlib import Path

RE_VAR_TYPE = r"[a-zA-Z0-9_]*\s*: "
RE_VAR_ASSIGN = r"[a-zA-Z0-9_]*\s*= "


def read_stubfile_toplevel(stubfile: Path) -> Set[str]:
    # read the stubfile into a list of lines
    with open(stubfile, "r") as f:
        lines = f.readlines()
    # remove the newlines
    lines = [line.rstrip("\n") for line in lines]

    # # find all lines that start with the regex "def [a-zA-Z0-9_]*\("
    # deflines = []
    # for line in lines:
    #     if re.match(r"def [a-zA-Z0-9_]*\s*\(", line):
    #         deflines.append(line)

    # # find all lines that start with the regex "class [a-zA-Z0-9_]*\("
    # classlines = []
    # for line in lines:
    #     if re.match(r"class [a-zA-Z0-9_]*\s*\(", line):
    #         classlines.append(line)

    # find all lines that start with the regex "[a-zA-Z0-9_]*: "
    typelines = []
    for line in lines:
        if re.match(RE_VAR_TYPE, line):
            typelines.append(line)

    # find all lines that start with the regex "[a-zA-Z0-9_]* = "
    varlines = []
    for line in lines:
        if re.match(RE_VAR_ASSIGN, line):
            varlines.append(line)

    toplevel_vars = set()
    toplevel_vars = toplevel_vars.union([v.split(":")[0].strip() for v in typelines])
    toplevel_vars = toplevel_vars.union([v.split("=")[0].strip() for v in varlines])
    return toplevel_vars

In [6]:
def patch_stdlib_toplevel(stubfile: Path, toplevel_vars: Set[str]) -> int:
    # read the stubfile into a list of lines
    with open(stubfile, "r") as f:
        lines = f.readlines()
    # remove the newlines
    updated = 0
    for i, line in enumerate(lines):
        # comment out the lines that are not toplevel vars
        if re.match(RE_VAR_TYPE, line):
            varname = line.split(":")[0].strip()
            if not varname in toplevel_vars and varname[0] != "_":
                lines[i] = "# " + line
                updated += 1
        elif re.match(RE_VAR_ASSIGN, line):
            varname = line.split("=")[0].strip()
            if not varname in toplevel_vars and varname[0] != "_":
                lines[i] = "# " + line
                updated += 1

    # write the stubfile back
    with open(stubfile, "w") as f:
        f.writelines(lines)

    return updated

In [8]:
stubfile = "C:\\develop\\MyPython\\micropython-stubs\\publish\\micropython-latest-esp32-stubs\\sys.pyi"
stdlib_stub = Path(
    "C:\\develop\\MyPython\\micropython-stubs\\publish\\micropython-stdlib-stubs\\stdlib\\sys\\__init__.pyi"
)
toplevel_vars = read_stubfile_toplevel(stubfile)
print(toplevel_vars)

patch_stdlib_toplevel(stdlib_stub, toplevel_vars)

{'stderr', 'argv', 'implementation', 'ps2', 'stdin', 'stdout', 'platform', 'maxsize', 'byteorder', 'ps1', 'modules', 'version', 'version_info', 'path'}


27